# **Control Flow Functions and Views**

#### **Executive Roundtable Challenge**

**Duration:** ~1 hour  
**Assumptions:** Learners have already completed lessons on SQL `CASE`, `COALESCE`, `NULLIF`, `IF`, and SQL Views. 

   

You’re a **Data Analyst** at a popular restaurant chain.

**Today, you’re at the executive roundtable with:**  
- The CEO  
- The Head Chef  
- The Marketing Director  
- The CFO

**On the table:** your laptop, the full recipe data, your analytical skills, and AI as your smart assistant.


**Your job:** write smart SQL queries, generate insightful views, and collaborate with AI to deliver answers, not shortcuts or guesswork.


![Executive Roundtable](https://raw.githubusercontent.com/DesmondMokhali/sql-controlflow-views/main/Round%20table.jpg)

### **Learning Outcomes**

**By the end of this module, students should be able to:**

- Understand and use SQL control flow functions (e.g., `CASE`, `IF`, `COALESCE`, `NULLIF`) and views in practical scenarios.  
- Apply these SQL concepts in data-rich, story-driven tasks that simulate real-world challenges.  
- Use AI critically and ethically, not as a shortcut, but to enhance reasoning, generate ideas, and refine problem-solving approaches.  
- Write effective, context-aware AI prompts to support coding, debugging, and explanation processes.

---

### **What’s AI Got To Do With It?**

**AI tools (like ([ChatGPT](https://chat.openai.com), [Claude](https://claude.ai)) can assist you to:**  
- Write and debug SQL queries  
- Explain your code and logic  
- Provide feedback and suggestions to improve your solutions

**BUT**, AI will not learn for you. Poorly crafted prompts lead to unreliable or irrelevant answers.

Let’s master the art of prompting to get the best results and truly enhance your learning. 💪



### **Mastering AI Prompting: Examples of Effective Prompts**

Imagine you want to categorize recipes based on their preparation time.

**❌ Poor Prompt:**  
Write a SQL query with CASE.

**⚠️ Improved Prompt:**  
Write a SQL query that uses CASE to group recipes.

**✅ Excellent Prompt:**  
Write a SQL query that uses CASE to classify recipes as 'Quick' if preparation time is under 20 minutes,  
'Moderate' if between 20 and 45 minutes, and 'Long' if over 45 minutes.

### Why this works:

- Provides **clear context** by specifying the dataset and column (recipes and preparation time)  
- Defines **explicit conditions** and meaningful categories  
- Specifies the **expected output format**, guiding precise query construction

---

### 📂 **Dataset Overview**

We’re using the **Food.com Recipes Dataset**, sourced from Kaggle:  
**Source:** [Food.com Recipes and User Interactions – Kaggle](https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions)

**Format:** CSV

### 🔑 Key Columns:

- **RecipeId:** Unique recipe identifier  
- **Name:** Recipe name  
- **AuthorId / AuthorName:** Creator information  
- **CookTime, PrepTime, TotalTime:** Time details (in minutes)  
- **DatePublished:** Publication date  
- **Description:** Short description of the recipe  
- **RecipeCategory / Keywords:** Tags and category labels (e.g., vegan, dessert)  
- **RecipeIngredientQuantities / RecipeIngredientParts:** Ingredients and their quantities  
- **AggregatedRating:** Average rating score  
- **ReviewCount:** Number of reviews (popularity proxy)  
- **Calories, FatContent, SaturatedFatContent, CholesterolContent, SodiumContent, CarbohydrateContent, FiberContent, SugarContent, ProteinContent:** Nutritional information  
- **RecipeServings / RecipeYield:** Portion sizes  
- **RecipeInstructions:** Step-by-step cooking instructions

For full field definitions and dataset details, refer to the Kaggle dataset page linked above.

### 🔧 **Data Preparation**

For this module, we have randomly selected **100 recipes** from the full dataset to keep the workload manageable and focused on a diverse, practical subset.

This sample maintains a rich variety of recipes for effective learning while ensuring efficient processing in SQL queries and notebook operations.

---

### ⚙️ **Environment Setup**

This notebook uses **Pandas**, **SQLite**, and **ipywidgets** to create an interactive and clean learning experience.  
We’ll set up everything here so that the rest of the notebook runs smoothly without extra installations.  

- **pandas** → data manipulation & analysis  
- **sqlite3** → lightweight SQL database for queries  
- **ipywidgets** → interactivity (dropdowns, buttons, etc.)  
- **IPython.display** → nice Markdown/text output  
- **matplotlib & seaborn** → simple plots and visuals  

> 💡 If you ever restart the kernel, just re-run this cell to restore the environment.


In [ ]:
# Install required packages (quiet mode)
!pip install ipywidgets pandas matplotlib seaborn requests --quiet

# Imports
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import requests
from io import StringIO

# Notebook settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
plt.style.use("seaborn-v0_8")  # cleaner plots

# Helper function to display Markdown easily
def show_md(text):
    display(Markdown(text))

# Environment ready
show_md("**Environment setup complete!** You can now run all cells below.")



### **Loading and Inspecting the Famous Recipes Dataset**

In [ ]:
# Download CSV from Google Drive
file_id = "1WxwDmc_GtBGlobm0yKx9wrGbo95G28x9"
url = f"https://drive.google.com/uc?export=download&id={file_id}"

response = requests.get(url)
response.raise_for_status()

# Read CSV into pandas DataFrame
csv_data = StringIO(response.text)
df = pd.read_csv(csv_data)

# Create in-memory SQLite DB
conn = sqlite3.connect(":memory:")
df.to_sql("Recipes", conn, index=False, if_exists="replace")
print(f"Dataset loaded into SQL with {len(df)} rows.")

# Show first 5 rows (compact)
display(Markdown("### First 5 rows (key columns only)"))
display(df.head())

With the data loaded, we’ll now apply SQL control flow and views to categorize and explore these recipes efficiently.

---

# **Question 1**



**Lesson:** SQL Control Flow Functions (CASE Statement Basics)

You’re tasked with classifying recipes from the **Breakfast** and **Beverages** categories.  
Because drinks typically take less time to prepare than food, your execs want **different prep time categories** for each:

- For **Breakfast recipes**:
  - **Quick:** under 30 minutes  
  - **Moderate:** 30 to 60 minutes  
  - **Long:** over 60 minutes  

- For **Beverages**:
  - **Quick:** under 10 minutes  
  - **Moderate:** 10 to 20 minutes  
  - **Long:** over 20 minutes  

### **Task 1**
**Write a SQL query that:**

1. Filters recipes to only those in the categories **Breakfast** and **Beverages**.  
2. Uses a `CASE` statement to classify recipes into the correct prep categories based on their category-specific thresholds.  
3. Shows the **top 5 recipes for each category (Breakfast Quick, Breakfast Moderate, Breakfast Long, Beverage Quick, Beverage Moderate, Beverage Long)** ordered by `TotalTime` descending.  
4. Returns columns: `Name`, `TotalTime`, `PrepCategory`, and `RecipeCategory`.

### **Draft your AI prompt to generate this query**

- Clearly specify the dataset and relevant columns: `Name`, `TotalTime`, and `RecipeCategory`.  
- Define the category-specific prep time thresholds explicitly.  
- Specify that the query should filter for Breakfast and Beverages only.  
- Explain the need to rank and limit the results to top 5 per category/prep level combination.  
- Mention the expected output columns.

*Use your favorite AI tool ([ChatGPT](https://chat.openai.com), [Claude](https://claude.ai), etc.) to craft the SQL query from this prompt.*


### **(I) Paste your AI prompt below, then run your AI tool externally.**

In [ ]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

# Textarea for AI prompt input
ai_prompt_input = widgets.Textarea(
    value='',
    placeholder='Enter your AI prompt here...',
    description='AI Prompt:',
    layout=widgets.Layout(width='100%', height='120px')
)
display(ai_prompt_input)

# Feedback output area for prompt validation
prompt_feedback = widgets.Output()
display(prompt_feedback)

# Button to acknowledge prompt submission
submit_prompt_button = widgets.Button(description="Prompt Entered ✓", button_style='success')

def on_submit_prompt_clicked(b):
    with prompt_feedback:
        clear_output()
        prompt = ai_prompt_input.value.strip()
        if not prompt:
            print("⚠️ Please enter your AI prompt before submitting.")
        elif len(prompt) < 150:
            print("⚠️ Your prompt is too short to be effective. Try adding more detail.")
        else:
            print("✅ Prompt looks good! Now run it with your AI tool externally and get the SQL query.")
            
submit_prompt_button.on_click(on_submit_prompt_clicked)
display(submit_prompt_button)


### **(II) Paste the AI-generated SQL query below to continue.**


In [ ]:
# Expected results markdown text
expected_results_md = """
### Expected Results Sample

| Name                   | TotalTime | PrepCategory | RecipeCategory |
|------------------------|-----------|--------------|----------------|
| Breakfast Casserole    | 65        | Long         | Breakfast      |
| Pancakes               | 45        | Moderate     | Breakfast      |
| Avocado Toast          | 20        | Quick        | Breakfast      |
| Fresh Orange Juice     | 15        | Moderate     | Beverages      |
| Banana Smoothie        | 8         | Quick        | Beverages      |
| Herbal Tea             | 25        | Long         | Beverages      |



*Your query should categorize recipes by `TotalTime` into `PrepCategory` as 'Quick' (<20), 'Moderate' (20-45), 'Long' (>45)*
"""

# Widget to toggle expected results
toggle_button = widgets.ToggleButton(
    value=False,
    description='Show Expected Results',
    button_style='info',
    tooltip='Click to show/hide expected results',
    icon='eye'
)

expected_results_output = widgets.Output()

def on_toggle_change(change):
    expected_results_output.clear_output()
    if change['new']:
        toggle_button.description = 'Hide Expected Results'
        with expected_results_output:
            display(Markdown(expected_results_md))
    else:
        toggle_button.description = 'Show Expected Results'

toggle_button.observe(on_toggle_change, names='value')

# SQL input widgets
query_input = widgets.Textarea(
    value='',
    placeholder='Paste your AI-generated SQL query here...',
    description='SQL Query:',
    layout=widgets.Layout(width='100%', height='120px')
)

run_button = widgets.Button(description='Run Query', button_style='success')
output = widgets.Output()

def on_run_button_clicked(b):
    with output:
        clear_output()
        query = query_input.value.strip()
        if not query:
            print("⚠️ Please enter your SQL query before submitting.")
            return
        if len(query) < 20:
            print("⚠️ Your SQL query seems too short. Please double-check your prompt or code.")
            return
        try:
            result = pd.read_sql_query(query, conn)
            display(result)
        except Exception as e:
            print(f"❌ SQL Error: {e}")

run_button.on_click(on_run_button_clicked)

# Explanation text area
explanation_input = widgets.Textarea(
    value='',
    placeholder='Write your explanation or reasoning for your SQL query here...',
    description='Your Explanation:',
    layout=widgets.Layout(width='100%', height='100px')
)

submit_explanation_btn = widgets.Button(description='Submit Explanation', button_style='primary')
explanation_output = widgets.Output()

def on_submit_explanation(b):
    with explanation_output:
        clear_output()
        explanation = explanation_input.value.strip()
        if not explanation:
            print("⚠️ Explanation cannot be empty. Please share your reasoning.")
        else:
            print("✅ Explanation submitted:")
            print(explanation)

submit_explanation_btn.on_click(on_submit_explanation)

# Display widgets in order
display(
    toggle_button, expected_results_output,
    query_input, run_button, output,
    explanation_input, submit_explanation_btn, explanation_output
)



### **Reflection:** 

Reflect on your experience using AI to help write and understand SQL control flow queries:

- How did AI assist you in framing the SQL `CASE` statement? Did it generate exactly what you expected or did you need to refine your prompt?
- Did you fully understand the AI-generated query? If not, how did you approach clarifying or debugging it?
- What challenges did you face when interpreting AI output or results from your SQL query?
- How did your understanding of conditional logic in SQL improve through interacting with AI?
- How can you leverage AI as a learning partner effectively without becoming overly reliant on it?



---

# **Question 2**

**Lesson:** COALESCE for replacing NULLs.

### **Task 2**

- Create a SQL view on this dataset that replaces missing Description values with **"Description not provided"** using `COALESCE`.
- Keep the difficulty classification logic from **Task 1**: classify recipes by `TotalTime` into categories like Easy/Medium/Hard using a `CASE` statement.

**Select these columns in your view:**
  
  - `Name`,  
  - `Description` (with `COALESCE` applied),  
  - `TotalTime`,
  - `PrepCategory` (created via `CASE` for difficulty),  
  - `RecipeCategory`.     



- Filter your view to include only recipes where `RecipeCategory` is `Breakfast` or `Beverages`.
- Use an **external AI tool** to prompt for explanations of different **SQL NULL-handling methods** (e.g.,  `COALESCE`, `IFNULL`, `ISNULL`, `CASE`).
- Compare the AI’s explanation with your own understanding and implementation, noting any differences or insights.

### **Example Expected SQL Query Output**

In [ ]:
# Expected output example markdown
expected_sql_output_md = """


| Name                   | Description                  | TotalTime | PrepCategory | RecipeCategory |
|------------------------|------------------------------|-----------|--------------|----------------|
| Picnic Cowboy Caviar   | Make and share this Picnic Cowboy Caviar recipe from Food.com. | 80        | Hard         | Breakfast      |
| Banilla Splash          | Make and share this Banilla Splash recipe from Food.com.       | 3         | Easy         | Beverages      |
| Some Missing Desc Recip | Description not provided      | 20        | Easy         | Breakfast      |

"""

toggle_sql_output = widgets.ToggleButton(
    value=False,
    description='Show Expected Query Output',
    button_style='info',
    icon='eye'
)
sql_output_box = widgets.Output()

def toggle_sql_output_change(change):
    sql_output_box.clear_output()
    if change['new']:
        toggle_sql_output.description = 'Hide Expected Query Output'
        with sql_output_box:
            display(Markdown(expected_sql_output_md))
    else:
        toggle_sql_output.description = 'Show Expected Query Output'

toggle_sql_output.observe(toggle_sql_output_change, names='value')

display(toggle_sql_output, sql_output_box)

# Student SQL input widget
student_query_input = widgets.Textarea(
    value='',
    placeholder='Write your SQL query here...',
    description='Your SQL Query:',
    layout=widgets.Layout(width='100%', height='120px')
)


### **(I) Enter Your SQL Query**

In [ ]:

# Widget to CREATE the view
create_view_input = widgets.Textarea(
    placeholder="Write your SQL to CREATE VIEW here...",
    layout=widgets.Layout(width="100%", height="100px")
)

create_view_btn = widgets.Button(description="Create View", button_style="info")
create_view_output = widgets.Output()

def on_create_view(b):
    create_view_output.clear_output()
    with create_view_output:
        sql = create_view_input.value.strip()
        if not sql:
            print("⚠️ No CREATE VIEW statement entered.")
            return
        try:
            conn.execute(sql)
            print("✅ View created successfully.")
        except Exception as e:
            print(f"❌ SQL Error: {e}")

create_view_btn.on_click(on_create_view)


# Widget to query the created view
student_query_input = widgets.Textarea(
    placeholder="Write your SQL query here (e.g., SELECT * FROM Task2_View)...",
    layout=widgets.Layout(width="100%", height="120px")
)

run_student_btn = widgets.Button(description="Run Your Query", button_style="success")
student_output = widgets.Output()

def run_query(query):
    student_output.clear_output()
    with student_output:
        try:
            if not query.strip():
                print("⚠️ No query entered.")
                return
            result = pd.read_sql_query(query, conn)
            if result.empty:
                print("⚠️ Query ran but returned no results.")
            else:
                display(result.head(10))
        except Exception as e:
            print(f"❌ SQL Error: {e}")

def on_run_student(b):
    run_query(student_query_input.value)

run_student_btn.on_click(on_run_student)


# Display both sections
display(
    widgets.VBox([
        widgets.Label("Create your view"),
        create_view_input, create_view_btn, create_view_output,
        widgets.Label("Query your view"),
        student_query_input, run_student_btn, student_output
    ])
)


### **(II) AI’s Feedback**

In [ ]:
# AI prompt for performance review

ai_feedback_input = widgets.Textarea(
    value='',
    placeholder='Paste or write AI feedback here...',
    description='AI Feedback:',
    layout=widgets.Layout(width='100%', height='120px')
)

submit_ai_feedback_btn = widgets.Button(description='Submit AI Feedback', button_style='primary')
ai_feedback_output = widgets.Output()

def on_submit_ai_feedback(b):
    ai_feedback_output.clear_output()
    with ai_feedback_output:
        if not ai_feedback_input.value.strip():
            print("⚠️ Please enter AI feedback before submitting.")
        else:
            print("✅ AI feedback submitted. Well done considering performance!")

submit_ai_feedback_btn.on_click(on_submit_ai_feedback)


display(ai_feedback_input, submit_ai_feedback_btn, ai_feedback_output)

#### **Reflection**

Compare your query to the AI’s explanation. Where did your approach match or differ? What did you learn about handling `NULLs` with `COALESCE` versus `CASE`?"


---

# **Question 3**

**Lesson:** Mixing multiple control flow functions.

### **Task 3** 
The **CFO**  wants to see the average cost estimate per difficulty level, where  `cost estimate` = `PrepTime` + `CookTime minutes`, and **"Unknown"** difficulty if cooking time is missing.

**Write a query that:**

- Uses `CASE` for difficulty classification based on `TotalTime`, with missing values showing **"Unknown"**
- Uses `COALESCE` to handle missing cooking times
- Calculates average cost estimate (`PrepTime` + `CookTime`) per difficulty level
- Write an AI prompt that clearly requests a SQL query to solve the same problem.  
- Run both your query and the AI-generated query to compare results.  
- Reflect on differences, AI usefulness, and challenges you faced. 



###  **(I) Write your SQL query below.**


### **Example Expected Output**

In [ ]:
# Example expected output for Task 3
expected_results_md = """
| Difficulty | AvgCostEstimate | RecipeCount |
|--------------|----------------|-------------|
| Easy         | 22.5           | 18          |
| Medium       | 45.3           | 25          |
| Hard         | 78.9           | 10          |
| Unknown      | 40.0           | 5           |
"""

toggle_expected = widgets.ToggleButton(
    value=False,
    description='Show Expected Output',
    button_style='info',
    icon='eye'
)
expected_output_box = widgets.Output()

def toggle_expected_change(change):
    expected_output_box.clear_output()
    if change['new']:
        toggle_expected.description = 'Hide Expected Output'
        with expected_output_box:
            display(Markdown(expected_results_md))
    else:
        toggle_expected.description = 'Show Expected Output'

toggle_expected.observe(toggle_expected_change, names='value')

display(toggle_expected, expected_output_box)


# Student SQL input widget
student_query_input = widgets.Textarea(
    value='',
    placeholder='Write your SQL query here...',
    description='SQL Query:',
    layout=widgets.Layout(width='100%', height='180px')
)

run_student_btn = widgets.Button(description='Run Your Query', button_style='success')
student_output = widgets.Output()


# Function to run student's query
def run_student_query(b):
    student_output.clear_output()
    with student_output:
        query = student_query_input.value.strip()
        if not query:
            print("⚠️ Please enter your SQL query.")
            return
        if len(query) < 20:
            print("⚠️ Query too short or incomplete.")
            return
        try:
            result = pd.read_sql_query(query, conn)
            if result.empty:
                print("⚠️ Query returned no results. Check filters or grouping conditions.")
            else:
                display(result.head(10))
        except Exception as e:
            print(f"❌ SQL Error: {e}")

run_student_btn.on_click(run_student_query)

display(student_query_input, run_student_btn, student_output)


### **(II) Write your AI prompt.**

In [ ]:
# Example instructions
ai_prompt_instructions = """
**Example AI prompt:**  

- I want to calculate the average `PrepTime` + `CookTime` per difficulty level in SQLite.
 - Use a `CASE` statement to classify difficulty (Easy, Medium, Hard, "Unknown" if TotalTime is missing).
- Use `COALESCE` to handle missing `PrepTime` or `CookTime`.
- Explain the approach.
"""

# Toggle to show/hide example
toggle_ai_prompt = widgets.ToggleButton(
    value=False,
    description='Show Example AI Prompt',
    button_style='info',
    icon='eye'
)
ai_prompt_box = widgets.Output()

def toggle_ai_prompt_change(change):
    ai_prompt_box.clear_output()
    if change['new']:
        toggle_ai_prompt.description = 'Hide Example AI Prompt'
        with ai_prompt_box:
            display(Markdown(ai_prompt_instructions))
    else:
        toggle_ai_prompt.description = 'Show Example AI Prompt'

toggle_ai_prompt.observe(toggle_ai_prompt_change, names='value')
display(toggle_ai_prompt, ai_prompt_box)

# AI prompt input widget
ai_prompt_input = widgets.Textarea(
    value='',
    placeholder='Paste your AI-generated query here...',
    description='AI SQL Query:',
    layout=widgets.Layout(width='100%', height='120px')
)

# AI Query run button & output
run_ai_btn = widgets.Button(description='Run AI Query', button_style='info')
ai_output = widgets.Output()

def run_query(query, output_widget):
    output_widget.clear_output()
    with output_widget:
        try:
            if not query.strip():
                print("⚠️ No query entered.")
                return
            if len(query.strip()) < 20:
                print("⚠️ Query too short or incomplete.")
                return
            result = pd.read_sql_query(query, conn)
            if result.empty:
                print("⚠️ Query ran but returned no results.")
            else:
                display(result.head(10))
        except Exception as e:
            print(f"❌ SQL Error: {e}")

def on_run_ai(b):
    ai_output.clear_output()
    with ai_output:
        print("⚠️ Please paste your AI-generated SQL query into the 'Your SQL Query' box and run it.") 

run_ai_btn.on_click(on_run_ai)

display(ai_prompt_input, run_ai_btn, ai_output)


####  **Reflection**

- How did using `CASE` and `COALESCE` together help handle missing `TotalTime` or `CookTime` values?
- Compare your query results with the AI-generated query. Were there any differences in logic, output, or handling of missing data?
- Which approach (your own or AI-generated) was easier to reason about, and why?
- What challenges did you encounter when combining multiple control flow functions, and how did you resolve them?
- What did this task teach you about using AI as a collaborator versus relying solely on your own SQL reasoning?

---

# **Question 4**

**Lesson:**  Creating and updating SQL views.

### **Task 4** 

The **CEO**  wants a permanent recipe_overview view combining difficulty, null-safe descriptions, and average ratings.

Create a persistent view using logic from previous tasks.

**Your task:** 

- Create a persistent **SQL view**  using the logic from **previous tasks** .

**Include at least:** 

- One control flow function (`CASE`, `IF`, etc.)
- One null-handling function (`COALESCE`, `NULLIF`, etc.)
- Use an external AI tool to draft a **CREATE VIEW**  statement, then compare it with your own implementation. 
- Check whether the AI’s draft includes all required column.
- Use `COALESCE()` (or another null-handling function) to replace missing descriptions.
- Classify recipes as **Easy**, **Medium**, or **Hard** based on `TotalTime` using a control flow function (`CASE`).
- Calculate the average rating per difficulty level.

**Hints / Approach**

- Think about **which columns can have NULL values** and use `COALESCE()` to handle them.
- Use a **CASE statement** to assign `PrepCategory` based on `TotalTime`.
- Aggregate ratings using **AVG()** grouped by difficulty or recipe.
- Combine all logic into a **persistent view** so the data can be queried easily by other analysts or dashboards.
  
    
  
⚠️ **Do not just copy-paste a query.**  
> Focus on planning your approach:  
> - Identify the transformations needed  
> - Decide where null handling is required  
> - Determine how to classify and aggregate

### **(II) Write your AI prompt.**

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import pandas as pd

# ===============================
# Expected Results Markdown
# ===============================
expected_results_md = """
### Expected Results Sample

| Name                       | Description                 | TotalTime | PrepCategory | RecipeCategory | AvgRating |
|----------------------------|----------------------------|-----------|--------------|----------------|-----------|
| Picnic Cowboy Caviar       | Make and share this Picnic | 80        | Medium       | Black Beans    | 4.0       |
| Turkey Schnitzel With Tots | Make and share this Turkey | 40        | Easy         | Poultry        | 5.3       |
| Banilla Splash             | Make and share this Banilla| 6         | Easy         | Beverages      | 5.0       |
"""

# Toggle button for expected results
toggle_button = widgets.ToggleButton(
    value=False,
    description='Show Expected Results',
    button_style='info',
    tooltip='Click to show/hide expected results',
    icon='eye'
)
expected_results_output = widgets.Output()

def on_toggle_change(change):
    expected_results_output.clear_output()
    if change['new']:
        toggle_button.description = 'Hide Expected Results'
        with expected_results_output:
            display(Markdown(expected_results_md))
    else:
        toggle_button.description = 'Show Expected Results'

toggle_button.observe(on_toggle_change, names='value')
display(toggle_button, expected_results_output)


# ===============================
# Example AI Prompt
# ===============================
ai_prompt_instructions = """
**Example prompt:**  

I want to create a persistent SQL view called `recipe_overview` in SQLite that summarizes recipes.

**Requirements**
- Include **Name**, **Description**, **TotalTime**, **PrepCategory** (difficulty via `CASE`), **RecipeCategory**, and **AggregatedRating**.
- Use **`COALESCE`** (or another null-handling function) to replace missing descriptions.
- Classify recipes by difficulty (**Easy**, **Medium**, **Hard**) using a control flow function.
- Show how to calculate the **average rating per difficulty level**.

**Approach**
Explain your approach without giving me a copy-paste query.
"""

toggle_ai_prompt = widgets.ToggleButton(
    value=False,
    description='Show Example AI Prompt',
    button_style='info',
    icon='eye'
)
ai_prompt_box = widgets.Output()

def toggle_ai_prompt_change(change):
    ai_prompt_box.clear_output()
    if change['new']:
        toggle_ai_prompt.description = 'Hide Example AI Prompt'
        with ai_prompt_box:
            display(Markdown(ai_prompt_instructions))
    else:
        toggle_ai_prompt.description = 'Show Example AI Prompt'

toggle_ai_prompt.observe(toggle_ai_prompt_change, names='value')
display(toggle_ai_prompt, ai_prompt_box)


In [ ]:
# Widget to CREATE the view
create_view_input = widgets.Textarea(
    placeholder="Write your SQL to CREATE VIEW here...",
    layout=widgets.Layout(width="100%", height="100px")
)

create_view_btn = widgets.Button(description="Create View", button_style="info")
create_view_output = widgets.Output()

def on_create_view(b):
    create_view_output.clear_output()
    with create_view_output:
        sql = create_view_input.value.strip()
        if not sql:
            print("⚠️ No CREATE VIEW statement entered.")
            return
        try:
            conn.execute(sql)
            print("✅ View created successfully.")
        except Exception as e:
            print(f"❌ SQL Error: {e}")

create_view_btn.on_click(on_create_view)


# Widget to query the created view
student_query_input = widgets.Textarea(
    placeholder="Write your SQL query here (e.g., SELECT * FROM Task2_View)...",
    layout=widgets.Layout(width="100%", height="120px")
)

run_student_btn = widgets.Button(description="Run Your Query", button_style="success")
student_output = widgets.Output()

def run_query(query):
    student_output.clear_output()
    with student_output:
        try:
            if not query.strip():
                print("⚠️ No query entered.")
                return
            result = pd.read_sql_query(query, conn)
            if result.empty:
                print("⚠️ Query ran but returned no results.")
            else:
                display(result.head(10))
        except Exception as e:
            print(f"❌ SQL Error: {e}")

def on_run_student(b):
    run_query(student_query_input.value)

run_student_btn.on_click(on_run_student)


# Display both sections
display(
    widgets.VBox([
        widgets.Label("Create your view"),
        create_view_input, create_view_btn, create_view_output,
        widgets.Label("Query your view"),
        student_query_input, run_student_btn, student_output
    ])
)




#### **Reflection**

- How did you verify that the AI’s draft included all required columns and logic?
- Did the AI’s use of `CASE` and `COALESCE` match your expectations? If not, what did you change?
- Was the AI’s clause order and syntax compatible with SQLite specifically?
- If the AI’s solution ran without errors but gave unexpected results, what debugging steps did you take?
- What’s the risk of over-relying on AI output for multi-requirement SQL queries?




---

# **Question 5**

**Lesson:** Complex Views with Multiple Control Flow Layers

### **Task 5** 

The executive team wants a single permanent SQL view that satisfies the needs of all four stakeholders:

- **Head Chef** — Classify each recipe’s difficulty level.  
- **Marketing Director** — Replace missing descriptions with a placeholder.  
- **CFO** — Label each recipe as `"Budget"` or `"Premium"` based on cost thresholds.  
- **CEO** — Add an `"Attention Needed"` flag for recipes with low ratings **and** missing descriptions.  

**Task**
- Create **one persistent view** that includes **all four logic layers**.
- Use **at least two distinct `CASE` statements** (they can be nested or sequential).
- Incorporate a **null-handling function** where applicable.
  

> ⚠️ **Note:** Focus on structuring your logic so each stakeholder’s requirement is met in a single query.



### **Expected Output Sample for `executive_report` View**

In [ ]:
# Expected output markdown (example for Task 5's view)
expected_output_md = """


| Name                           | Description                          | TotalTime | PrepCategory | RecipeCategory | CostLabel | AttentionNeeded | AvgRating |
| ------------------------------ | ------------------------------------ | --------- | ------------ | -------------- | --------- | --------------- | --------- |
| Picnic Cowboy Caviar           | Make and share this Picnic ...       | 80        | Medium       | Black Beans    | Budget    | No              | 4.0       |
| Turkey Schnitzel With Tots ... | Make and share this Turkey ...       | 40        | Easy         | Poultry        | Premium   | No              | 5.3       |
| Banilla Splash                 | Make and share this Banilla ...      | 6         | Easy         | Beverages      | Budget    | No              | 5.0       |
| Mystery Smoothie               | NULL                                 | 15        | Easy         | Beverages      | Budget    | Yes             | 3.2       |
| Vegan Lasagna                  | Make and share this Vegan Lasagna... | 90        | Hard         | Vegetarian     | Premium   | No              | 4.5       |

"""

expected_output_toggle = widgets.ToggleButton(
    value=False,
    description='Show Expected Output',
    button_style='info',
    icon='eye'
)

expected_output_display = widgets.Output()

def toggle_expected_output(change):
    expected_output_display.clear_output()
    if change['new']:
        expected_output_toggle.description = 'Hide Expected Output'
        with expected_output_display:
            display(Markdown(expected_output_md))
    else:
        expected_output_toggle.description = 'Show Expected Output'

expected_output_toggle.observe(toggle_expected_output, names='value')

display(expected_output_toggle, expected_output_display)

In [ ]:
# Widget to CREATE the view
create_view_input = widgets.Textarea(
    placeholder="Write your SQL to CREATE VIEW here...",
    layout=widgets.Layout(width="100%", height="100px")
)

create_view_btn = widgets.Button(description="Create View", button_style="info")
create_view_output = widgets.Output()

def on_create_view(b):
    create_view_output.clear_output()
    with create_view_output:
        sql = create_view_input.value.strip()
        if not sql:
            print("⚠️ No CREATE VIEW statement entered.")
            return
        try:
            conn.execute(sql)
            print("✅ View created successfully.")
        except Exception as e:
            print(f"❌ SQL Error: {e}")

create_view_btn.on_click(on_create_view)


# Widget to query the created view
student_query_input = widgets.Textarea(
    placeholder="Write your SQL query here (e.g., SELECT * FROM Task2_View)...",
    layout=widgets.Layout(width="100%", height="120px")
)

run_student_btn = widgets.Button(description="Run Your Query", button_style="success")
student_output = widgets.Output()

def run_query(query):
    student_output.clear_output()
    with student_output:
        try:
            if not query.strip():
                print("⚠️ No query entered.")
                return
            result = pd.read_sql_query(query, conn)
            if result.empty:
                print("⚠️ Query ran but returned no results.")
            else:
                display(result.head(10))
        except Exception as e:
            print(f"❌ SQL Error: {e}")

def on_run_student(b):
    run_query(student_query_input.value)

run_student_btn.on_click(on_run_student)


# Display both sections
display(
    widgets.VBox([
        widgets.Label("Create your view"),
        create_view_input, create_view_btn, create_view_output,
        widgets.Label("Query your view"),
        student_query_input, run_student_btn, student_output
    ])
)




# **Control Flow Functions and Views**

#### **Executive Roundtable Challenge**

**Duration:** ~1 hour  
**Assumptions:** Learners have already completed lessons on SQL `CASE`, `COALESCE`, `NULLIF`, `IF`, and SQL Views. 

   

You’re a **Data Analyst** at a popular restaurant chain.

**Today, you’re at the executive roundtable with:**  
- The CEO  
- The Head Chef  
- The Marketing Director  
- The CFO

**On the table:** your laptop, the full recipe data, your analytical skills, and AI as your smart assistant.


**Your job:** write smart SQL queries, generate insightful views, and collaborate with AI to deliver answers, not shortcuts or guesswork.


![Executive Roundtable](https://raw.githubusercontent.com/DesmondMokhali/sql-controlflow-views/main/Round%20table.jpg)

### **Learning Outcomes**

**By the end of this module, students should be able to:**

- Understand and use SQL control flow functions (e.g., `CASE`, `IF`, `COALESCE`, `NULLIF`) and views in practical scenarios.  
- Apply these SQL concepts in data-rich, story-driven tasks that simulate real-world challenges.  
- Use AI critically and ethically, not as a shortcut, but to enhance reasoning, generate ideas, and refine problem-solving approaches.  
- Write effective, context-aware AI prompts to support coding, debugging, and explanation processes.

---

### **What’s AI Got To Do With It?**

**AI tools (like ([ChatGPT](https://chat.openai.com), [Claude](https://claude.ai)) can assist you to:**  
- Write and debug SQL queries  
- Explain your code and logic  
- Provide feedback and suggestions to improve your solutions

**BUT**, AI will not learn for you. Poorly crafted prompts lead to unreliable or irrelevant answers.

Let’s master the art of prompting to get the best results and truly enhance your learning. 💪



### **Mastering AI Prompting: Examples of Effective Prompts**

Imagine you want to categorize recipes based on their preparation time.

**❌ Poor Prompt:**  
Write a SQL query with CASE.

**⚠️ Improved Prompt:**  
Write a SQL query that uses CASE to group recipes.

**✅ Excellent Prompt:**  
Write a SQL query that uses CASE to classify recipes as 'Quick' if preparation time is under 20 minutes,  
'Moderate' if between 20 and 45 minutes, and 'Long' if over 45 minutes.

### Why this works:

- Provides **clear context** by specifying the dataset and column (recipes and preparation time)  
- Defines **explicit conditions** and meaningful categories  
- Specifies the **expected output format**, guiding precise query construction

---

### 📂 **Dataset Overview**

We’re using the **Food.com Recipes Dataset**, sourced from Kaggle:  
**Source:** [Food.com Recipes and User Interactions – Kaggle](https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions)

**Format:** CSV

### 🔑 Key Columns:

- **RecipeId:** Unique recipe identifier  
- **Name:** Recipe name  
- **AuthorId / AuthorName:** Creator information  
- **CookTime, PrepTime, TotalTime:** Time details (in minutes)  
- **DatePublished:** Publication date  
- **Description:** Short description of the recipe  
- **RecipeCategory / Keywords:** Tags and category labels (e.g., vegan, dessert)  
- **RecipeIngredientQuantities / RecipeIngredientParts:** Ingredients and their quantities  
- **AggregatedRating:** Average rating score  
- **ReviewCount:** Number of reviews (popularity proxy)  
- **Calories, FatContent, SaturatedFatContent, CholesterolContent, SodiumContent, CarbohydrateContent, FiberContent, SugarContent, ProteinContent:** Nutritional information  
- **RecipeServings / RecipeYield:** Portion sizes  
- **RecipeInstructions:** Step-by-step cooking instructions

For full field definitions and dataset details, refer to the Kaggle dataset page linked above.

### 🔧 **Data Preparation**

For this module, we have randomly selected **100 recipes** from the full dataset to keep the workload manageable and focused on a diverse, practical subset.

This sample maintains a rich variety of recipes for effective learning while ensuring efficient processing in SQL queries and notebook operations.

---

### ⚙️ **Environment Setup**

This notebook uses **Pandas**, **SQLite**, and **ipywidgets** to create an interactive and clean learning experience.  
We’ll set up everything here so that the rest of the notebook runs smoothly without extra installations.  

- **pandas** → data manipulation & analysis  
- **sqlite3** → lightweight SQL database for queries  
- **ipywidgets** → interactivity (dropdowns, buttons, etc.)  
- **IPython.display** → nice Markdown/text output  
- **matplotlib & seaborn** → simple plots and visuals  

> 💡 If you ever restart the kernel, just re-run this cell to restore the environment.


In [ ]:
# Install required packages (quiet mode)
!pip install ipywidgets pandas matplotlib seaborn requests --quiet

# Imports
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import requests
from io import StringIO

# Notebook settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
plt.style.use("seaborn-v0_8")  # cleaner plots

# Helper function to display Markdown easily
def show_md(text):
    display(Markdown(text))

# Environment ready
show_md("**Environment setup complete!** You can now run all cells below.")



### **Loading and Inspecting the Famous Recipes Dataset**

https://drive.google.com/file/d/1WxwDmc_GtBGlobm0yKx9wrGbo95G28x9/view?usp=sharing

In [ ]:
# Download CSV from Google Drive
file_id = "1WxwDmc_GtBGlobm0yKx9wrGbo95G28x9"
url = f"https://drive.google.com/uc?export=download&id={file_id}"

response = requests.get(url)
response.raise_for_status()

# Read CSV into pandas DataFrame
csv_data = StringIO(response.text)
df = pd.read_csv(csv_data)

# Create in-memory SQLite DB
conn = sqlite3.connect(":memory:")
df.to_sql("Recipes", conn, index=False, if_exists="replace")
print(f"Dataset loaded into SQL with {len(df)} rows.")

# Show first 5 rows (compact)
display(Markdown("### First 5 rows (key columns only)"))
display(df.head())

With the data loaded, we’ll now apply SQL control flow and views to categorize and explore these recipes efficiently.

---

# **Question 1**



**Lesson:** SQL Control Flow Functions (CASE Statement Basics)

You’re tasked with classifying recipes from the **Breakfast** and **Beverages** categories.  
Because drinks typically take less time to prepare than food, your execs want **different prep time categories** for each:

- For **Breakfast recipes**:
  - **Quick:** under 30 minutes  
  - **Moderate:** 30 to 60 minutes  
  - **Long:** over 60 minutes  

- For **Beverages**:
  - **Quick:** under 10 minutes  
  - **Moderate:** 10 to 20 minutes  
  - **Long:** over 20 minutes  

### **Task 1**
**Write a SQL query that:**

1. Filters recipes to only those in the categories **Breakfast** and **Beverages**.  
2. Uses a `CASE` statement to classify recipes into the correct prep categories based on their category-specific thresholds.  
3. Shows the **top 5 recipes for each category (Breakfast Quick, Breakfast Moderate, Breakfast Long, Beverage Quick, Beverage Moderate, Beverage Long)** ordered by `TotalTime` descending.  
4. Returns columns: `Name`, `TotalTime`, `PrepCategory`, and `RecipeCategory`.

### **Draft your AI prompt to generate this query**

- Clearly specify the dataset and relevant columns: `Name`, `TotalTime`, and `RecipeCategory`.  
- Define the category-specific prep time thresholds explicitly.  
- Specify that the query should filter for Breakfast and Beverages only.  
- Explain the need to rank and limit the results to top 5 per category/prep level combination.  
- Mention the expected output columns.

*Use your favorite AI tool ([ChatGPT](https://chat.openai.com), [Claude](https://claude.ai), etc.) to craft the SQL query from this prompt.*


### **(I) Paste your AI prompt below, then run your AI tool externally.**

In [ ]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

# Textarea for AI prompt input
ai_prompt_input = widgets.Textarea(
    value='',
    placeholder='Enter your AI prompt here...',
    description='AI Prompt:',
    layout=widgets.Layout(width='100%', height='120px')
)
display(ai_prompt_input)

# Feedback output area for prompt validation
prompt_feedback = widgets.Output()
display(prompt_feedback)

# Button to acknowledge prompt submission
submit_prompt_button = widgets.Button(description="Prompt Entered ✓", button_style='success')

def on_submit_prompt_clicked(b):
    with prompt_feedback:
        clear_output()
        prompt = ai_prompt_input.value.strip()
        if not prompt:
            print("⚠️ Please enter your AI prompt before submitting.")
        elif len(prompt) < 150:
            print("⚠️ Your prompt is too short to be effective. Try adding more detail.")
        else:
            print("✅ Prompt looks good! Now run it with your AI tool externally and get the SQL query.")
            
submit_prompt_button.on_click(on_submit_prompt_clicked)
display(submit_prompt_button)


### **(II) Paste the AI-generated SQL query below to continue.**


In [ ]:
# Expected results markdown text
expected_results_md = """
### Expected Results Sample

| Name                   | TotalTime | PrepCategory | RecipeCategory |
|------------------------|-----------|--------------|----------------|
| Breakfast Casserole    | 65        | Long         | Breakfast      |
| Pancakes               | 45        | Moderate     | Breakfast      |
| Avocado Toast          | 20        | Quick        | Breakfast      |
| Fresh Orange Juice     | 15        | Moderate     | Beverages      |
| Banana Smoothie        | 8         | Quick        | Beverages      |
| Herbal Tea             | 25        | Long         | Beverages      |



*Your query should categorize recipes by `TotalTime` into `PrepCategory` as 'Quick' (<20), 'Moderate' (20-45), 'Long' (>45)*
"""

# Widget to toggle expected results
toggle_button = widgets.ToggleButton(
    value=False,
    description='Show Expected Results',
    button_style='info',
    tooltip='Click to show/hide expected results',
    icon='eye'
)

expected_results_output = widgets.Output()

def on_toggle_change(change):
    expected_results_output.clear_output()
    if change['new']:
        toggle_button.description = 'Hide Expected Results'
        with expected_results_output:
            display(Markdown(expected_results_md))
    else:
        toggle_button.description = 'Show Expected Results'

toggle_button.observe(on_toggle_change, names='value')

# SQL input widgets
query_input = widgets.Textarea(
    value='',
    placeholder='Paste your AI-generated SQL query here...',
    description='SQL Query:',
    layout=widgets.Layout(width='100%', height='120px')
)

run_button = widgets.Button(description='Run Query', button_style='success')
output = widgets.Output()

def on_run_button_clicked(b):
    with output:
        clear_output()
        query = query_input.value.strip()
        if not query:
            print("⚠️ Please enter your SQL query before submitting.")
            return
        if len(query) < 20:
            print("⚠️ Your SQL query seems too short. Please double-check your prompt or code.")
            return
        try:
            result = pd.read_sql_query(query, conn)
            display(result)
        except Exception as e:
            print(f"❌ SQL Error: {e}")

run_button.on_click(on_run_button_clicked)

# Explanation text area
explanation_input = widgets.Textarea(
    value='',
    placeholder='Write your explanation or reasoning for your SQL query here...',
    description='Your Explanation:',
    layout=widgets.Layout(width='100%', height='100px')
)

submit_explanation_btn = widgets.Button(description='Submit Explanation', button_style='primary')
explanation_output = widgets.Output()

def on_submit_explanation(b):
    with explanation_output:
        clear_output()
        explanation = explanation_input.value.strip()
        if not explanation:
            print("⚠️ Explanation cannot be empty. Please share your reasoning.")
        else:
            print("✅ Explanation submitted:")
            print(explanation)

submit_explanation_btn.on_click(on_submit_explanation)

# Display widgets in order
display(
    toggle_button, expected_results_output,
    query_input, run_button, output,
    explanation_input, submit_explanation_btn, explanation_output
)



### **Reflection:** 

Reflect on your experience using AI to help write and understand SQL control flow queries:

- How did AI assist you in framing the SQL `CASE` statement? Did it generate exactly what you expected or did you need to refine your prompt?
- Did you fully understand the AI-generated query? If not, how did you approach clarifying or debugging it?
- What challenges did you face when interpreting AI output or results from your SQL query?
- How did your understanding of conditional logic in SQL improve through interacting with AI?
- How can you leverage AI as a learning partner effectively without becoming overly reliant on it?



---

# **Question 2**

**Lesson:** COALESCE for replacing NULLs.

### **Task 2**

- Create a SQL view on this dataset that replaces missing Description values with **"Description not provided"** using `COALESCE`.
- Keep the difficulty classification logic from **Task 1**: classify recipes by `TotalTime` into categories like Easy/Medium/Hard using a `CASE` statement.

**Select these columns in your view:**
  
  - `Name`,  
  - `Description` (with `COALESCE` applied),  
  - `TotalTime`,
  - `PrepCategory` (created via `CASE` for difficulty),  
  - `RecipeCategory`.     



- Filter your view to include only recipes where `RecipeCategory` is `Breakfast` or `Beverages`.
- Use an **external AI tool** to prompt for explanations of different **SQL NULL-handling methods** (e.g.,  `COALESCE`, `IFNULL`, `ISNULL`, `CASE`).
- Compare the AI’s explanation with your own understanding and implementation, noting any differences or insights.

### **Example Expected SQL Query Output**

In [ ]:
# Expected output example markdown
expected_sql_output_md = """


| Name                   | Description                  | TotalTime | PrepCategory | RecipeCategory |
|------------------------|------------------------------|-----------|--------------|----------------|
| Picnic Cowboy Caviar   | Make and share this Picnic Cowboy Caviar recipe from Food.com. | 80        | Hard         | Breakfast      |
| Banilla Splash          | Make and share this Banilla Splash recipe from Food.com.       | 3         | Easy         | Beverages      |
| Some Missing Desc Recip | Description not provided      | 20        | Easy         | Breakfast      |

"""

toggle_sql_output = widgets.ToggleButton(
    value=False,
    description='Show Expected Query Output',
    button_style='info',
    icon='eye'
)
sql_output_box = widgets.Output()

def toggle_sql_output_change(change):
    sql_output_box.clear_output()
    if change['new']:
        toggle_sql_output.description = 'Hide Expected Query Output'
        with sql_output_box:
            display(Markdown(expected_sql_output_md))
    else:
        toggle_sql_output.description = 'Show Expected Query Output'

toggle_sql_output.observe(toggle_sql_output_change, names='value')

display(toggle_sql_output, sql_output_box)

# Student SQL input widget
student_query_input = widgets.Textarea(
    value='',
    placeholder='Write your SQL query here...',
    description='Your SQL Query:',
    layout=widgets.Layout(width='100%', height='120px')
)


### **(I) Enter Your SQL Query**

In [ ]:

# Widget to CREATE the view
create_view_input = widgets.Textarea(
    placeholder="Write your SQL to CREATE VIEW here...",
    layout=widgets.Layout(width="100%", height="100px")
)

create_view_btn = widgets.Button(description="Create View", button_style="info")
create_view_output = widgets.Output()

def on_create_view(b):
    create_view_output.clear_output()
    with create_view_output:
        sql = create_view_input.value.strip()
        if not sql:
            print("⚠️ No CREATE VIEW statement entered.")
            return
        try:
            conn.execute(sql)
            print("✅ View created successfully.")
        except Exception as e:
            print(f"❌ SQL Error: {e}")

create_view_btn.on_click(on_create_view)


# Widget to query the created view
student_query_input = widgets.Textarea(
    placeholder="Write your SQL query here (e.g., SELECT * FROM Task2_View)...",
    layout=widgets.Layout(width="100%", height="120px")
)

run_student_btn = widgets.Button(description="Run Your Query", button_style="success")
student_output = widgets.Output()

def run_query(query):
    student_output.clear_output()
    with student_output:
        try:
            if not query.strip():
                print("⚠️ No query entered.")
                return
            result = pd.read_sql_query(query, conn)
            if result.empty:
                print("⚠️ Query ran but returned no results.")
            else:
                display(result.head(10))
        except Exception as e:
            print(f"❌ SQL Error: {e}")

def on_run_student(b):
    run_query(student_query_input.value)

run_student_btn.on_click(on_run_student)


# Display both sections
display(
    widgets.VBox([
        widgets.Label("Create your view"),
        create_view_input, create_view_btn, create_view_output,
        widgets.Label("Query your view"),
        student_query_input, run_student_btn, student_output
    ])
)


### **(II) AI’s Feedback**

In [ ]:
# AI prompt for performance review

ai_feedback_input = widgets.Textarea(
    value='',
    placeholder='Paste or write AI feedback here...',
    description='AI Feedback:',
    layout=widgets.Layout(width='100%', height='120px')
)

submit_ai_feedback_btn = widgets.Button(description='Submit AI Feedback', button_style='primary')
ai_feedback_output = widgets.Output()

def on_submit_ai_feedback(b):
    ai_feedback_output.clear_output()
    with ai_feedback_output:
        if not ai_feedback_input.value.strip():
            print("⚠️ Please enter AI feedback before submitting.")
        else:
            print("✅ AI feedback submitted. Well done considering performance!")

submit_ai_feedback_btn.on_click(on_submit_ai_feedback)


display(ai_feedback_input, submit_ai_feedback_btn, ai_feedback_output)

#### **Reflection**

Compare your query to the AI’s explanation. Where did your approach match or differ? What did you learn about handling `NULLs` with `COALESCE` versus `CASE`?"


---

# **Question 3**

**Lesson:** Mixing multiple control flow functions.

### **Task 3** 
The **CFO**  wants to see the average cost estimate per difficulty level, where  `cost estimate` = `PrepTime` + `CookTime minutes`, and **"Unknown"** difficulty if cooking time is missing.

**Write a query that:**

- Uses `CASE` for difficulty classification based on `TotalTime`, with missing values showing **"Unknown"**
- Uses `COALESCE` to handle missing cooking times
- Calculates average cost estimate (`PrepTime` + `CookTime`) per difficulty level
- Write an AI prompt that clearly requests a SQL query to solve the same problem.  
- Run both your query and the AI-generated query to compare results.  
- Reflect on differences, AI usefulness, and challenges you faced. 



###  **(I) Write your SQL query below.**


### **Example Expected Output**

In [ ]:
# Example expected output for Task 3
expected_results_md = """
| Difficulty | AvgCostEstimate | RecipeCount |
|--------------|----------------|-------------|
| Easy         | 22.5           | 18          |
| Medium       | 45.3           | 25          |
| Hard         | 78.9           | 10          |
| Unknown      | 40.0           | 5           |
"""

toggle_expected = widgets.ToggleButton(
    value=False,
    description='Show Expected Output',
    button_style='info',
    icon='eye'
)
expected_output_box = widgets.Output()

def toggle_expected_change(change):
    expected_output_box.clear_output()
    if change['new']:
        toggle_expected.description = 'Hide Expected Output'
        with expected_output_box:
            display(Markdown(expected_results_md))
    else:
        toggle_expected.description = 'Show Expected Output'

toggle_expected.observe(toggle_expected_change, names='value')

display(toggle_expected, expected_output_box)


# Student SQL input widget
student_query_input = widgets.Textarea(
    value='',
    placeholder='Write your SQL query here...',
    description='SQL Query:',
    layout=widgets.Layout(width='100%', height='180px')
)

run_student_btn = widgets.Button(description='Run Your Query', button_style='success')
student_output = widgets.Output()


# Function to run student's query
def run_student_query(b):
    student_output.clear_output()
    with student_output:
        query = student_query_input.value.strip()
        if not query:
            print("⚠️ Please enter your SQL query.")
            return
        if len(query) < 20:
            print("⚠️ Query too short or incomplete.")
            return
        try:
            result = pd.read_sql_query(query, conn)
            if result.empty:
                print("⚠️ Query returned no results. Check filters or grouping conditions.")
            else:
                display(result.head(10))
        except Exception as e:
            print(f"❌ SQL Error: {e}")

run_student_btn.on_click(run_student_query)

display(student_query_input, run_student_btn, student_output)


### **(II) Write your AI prompt.**

In [ ]:
# Example instructions
ai_prompt_instructions = """
**Example AI prompt:**  

- I want to calculate the average `PrepTime` + `CookTime` per difficulty level in SQLite.
 - Use a `CASE` statement to classify difficulty (Easy, Medium, Hard, "Unknown" if TotalTime is missing).
- Use `COALESCE` to handle missing `PrepTime` or `CookTime`.
- Explain the approach.
"""

# Toggle to show/hide example
toggle_ai_prompt = widgets.ToggleButton(
    value=False,
    description='Show Example AI Prompt',
    button_style='info',
    icon='eye'
)
ai_prompt_box = widgets.Output()

def toggle_ai_prompt_change(change):
    ai_prompt_box.clear_output()
    if change['new']:
        toggle_ai_prompt.description = 'Hide Example AI Prompt'
        with ai_prompt_box:
            display(Markdown(ai_prompt_instructions))
    else:
        toggle_ai_prompt.description = 'Show Example AI Prompt'

toggle_ai_prompt.observe(toggle_ai_prompt_change, names='value')
display(toggle_ai_prompt, ai_prompt_box)

# AI prompt input widget
ai_prompt_input = widgets.Textarea(
    value='',
    placeholder='Paste your AI-generated query here...',
    description='AI SQL Query:',
    layout=widgets.Layout(width='100%', height='120px')
)

# AI Query run button & output
run_ai_btn = widgets.Button(description='Run AI Query', button_style='info')
ai_output = widgets.Output()

def run_query(query, output_widget):
    output_widget.clear_output()
    with output_widget:
        try:
            if not query.strip():
                print("⚠️ No query entered.")
                return
            if len(query.strip()) < 20:
                print("⚠️ Query too short or incomplete.")
                return
            result = pd.read_sql_query(query, conn)
            if result.empty:
                print("⚠️ Query ran but returned no results.")
            else:
                display(result.head(10))
        except Exception as e:
            print(f"❌ SQL Error: {e}")

def on_run_ai(b):
    ai_output.clear_output()
    with ai_output:
        print("⚠️ Please paste your AI-generated SQL query into the 'Your SQL Query' box and run it.") 

run_ai_btn.on_click(on_run_ai)

display(ai_prompt_input, run_ai_btn, ai_output)


####  **Reflection**

- How did using `CASE` and `COALESCE` together help handle missing `TotalTime` or `CookTime` values?
- Compare your query results with the AI-generated query. Were there any differences in logic, output, or handling of missing data?
- Which approach (your own or AI-generated) was easier to reason about, and why?
- What challenges did you encounter when combining multiple control flow functions, and how did you resolve them?
- What did this task teach you about using AI as a collaborator versus relying solely on your own SQL reasoning?

In [ ]:
# Reflection
reflection_input = widgets.Textarea(
    value='',
    placeholder='Write your reflection here...',
    description='Reflection:',
    layout=widgets.Layout(width='100%', height='150px')
)

reflection_submit_btn = widgets.Button(description='Submit Reflection', button_style='primary')
reflection_output = widgets.Output()

def on_reflection_submit(b):
    reflection_output.clear_output()
    with reflection_output:
        if not reflection_input.value.strip():
            print("⚠️ Please enter your reflection before submitting.")
        else:
            print("✅ Reflection submitted. Thanks for your thoughtful insights!")

reflection_submit_btn.on_click(on_reflection_submit)

display(reflection_input, reflection_submit_btn, reflection_output)

---

# **Question 4**

**Lesson:**  Creating and updating SQL views.

### **Task 4** 

The **CEO**  wants a permanent recipe_overview view combining difficulty, null-safe descriptions, and average ratings.

Create a persistent view using logic from previous tasks.

**Your task:** 

- Create a persistent **SQL view**  using the logic from **previous tasks** .

**Include at least:** 

- One control flow function (`CASE`, `IF`, etc.)
- One null-handling function (`COALESCE`, `NULLIF`, etc.)
- Use an external AI tool to draft a **CREATE VIEW**  statement, then compare it with your own implementation. 
- Check whether the AI’s draft includes all required column.
- Use `COALESCE()` (or another null-handling function) to replace missing descriptions.
- Classify recipes as **Easy**, **Medium**, or **Hard** based on `TotalTime` using a control flow function (`CASE`).
- Calculate the average rating per difficulty level.

**Hints / Approach**

- Think about **which columns can have NULL values** and use `COALESCE()` to handle them.
- Use a **CASE statement** to assign `PrepCategory` based on `TotalTime`.
- Aggregate ratings using **AVG()** grouped by difficulty or recipe.
- Combine all logic into a **persistent view** so the data can be queried easily by other analysts or dashboards.
  
    
  
⚠️ **Do not just copy-paste a query.**  
> Focus on planning your approach:  
> - Identify the transformations needed  
> - Decide where null handling is required  
> - Determine how to classify and aggregate

### **(II) Write your AI prompt.**

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import pandas as pd

# ===============================
# Expected Results Markdown
# ===============================
expected_results_md = """
### Expected Results Sample

| Name                       | Description                 | TotalTime | PrepCategory | RecipeCategory | AvgRating |
|----------------------------|----------------------------|-----------|--------------|----------------|-----------|
| Picnic Cowboy Caviar       | Make and share this Picnic | 80        | Medium       | Black Beans    | 4.0       |
| Turkey Schnitzel With Tots | Make and share this Turkey | 40        | Easy         | Poultry        | 5.3       |
| Banilla Splash             | Make and share this Banilla| 6         | Easy         | Beverages      | 5.0       |
"""

# Toggle button for expected results
toggle_button = widgets.ToggleButton(
    value=False,
    description='Show Expected Results',
    button_style='info',
    tooltip='Click to show/hide expected results',
    icon='eye'
)
expected_results_output = widgets.Output()

def on_toggle_change(change):
    expected_results_output.clear_output()
    if change['new']:
        toggle_button.description = 'Hide Expected Results'
        with expected_results_output:
            display(Markdown(expected_results_md))
    else:
        toggle_button.description = 'Show Expected Results'

toggle_button.observe(on_toggle_change, names='value')
display(toggle_button, expected_results_output)


# ===============================
# Example AI Prompt
# ===============================
ai_prompt_instructions = """
**Example prompt:**  

I want to create a persistent SQL view called `recipe_overview` in SQLite that summarizes recipes.

**Requirements**
- Include **Name**, **Description**, **TotalTime**, **PrepCategory** (difficulty via `CASE`), **RecipeCategory**, and **AggregatedRating**.
- Use **`COALESCE`** (or another null-handling function) to replace missing descriptions.
- Classify recipes by difficulty (**Easy**, **Medium**, **Hard**) using a control flow function.
- Show how to calculate the **average rating per difficulty level**.

**Approach**
Explain your approach without giving me a copy-paste query.
"""

toggle_ai_prompt = widgets.ToggleButton(
    value=False,
    description='Show Example AI Prompt',
    button_style='info',
    icon='eye'
)
ai_prompt_box = widgets.Output()

def toggle_ai_prompt_change(change):
    ai_prompt_box.clear_output()
    if change['new']:
        toggle_ai_prompt.description = 'Hide Example AI Prompt'
        with ai_prompt_box:
            display(Markdown(ai_prompt_instructions))
    else:
        toggle_ai_prompt.description = 'Show Example AI Prompt'

toggle_ai_prompt.observe(toggle_ai_prompt_change, names='value')
display(toggle_ai_prompt, ai_prompt_box)



In [ ]:
# Widget to CREATE the view
create_view_input = widgets.Textarea(
    placeholder="Write your SQL to CREATE VIEW here...",
    layout=widgets.Layout(width="100%", height="100px")
)

create_view_btn = widgets.Button(description="Create View", button_style="info")
create_view_output = widgets.Output()

def on_create_view(b):
    create_view_output.clear_output()
    with create_view_output:
        sql = create_view_input.value.strip()
        if not sql:
            print("⚠️ No CREATE VIEW statement entered.")
            return
        try:
            conn.execute(sql)
            print("✅ View created successfully.")
        except Exception as e:
            print(f"❌ SQL Error: {e}")

create_view_btn.on_click(on_create_view)


# Widget to query the created view
student_query_input = widgets.Textarea(
    placeholder="Write your SQL query here (e.g., SELECT * FROM Task2_View)...",
    layout=widgets.Layout(width="100%", height="120px")
)

run_student_btn = widgets.Button(description="Run Your Query", button_style="success")
student_output = widgets.Output()

def run_query(query):
    student_output.clear_output()
    with student_output:
        try:
            if not query.strip():
                print("⚠️ No query entered.")
                return
            result = pd.read_sql_query(query, conn)
            if result.empty:
                print("⚠️ Query ran but returned no results.")
            else:
                display(result.head(10))
        except Exception as e:
            print(f"❌ SQL Error: {e}")

def on_run_student(b):
    run_query(student_query_input.value)

run_student_btn.on_click(on_run_student)


# Display both sections
display(
    widgets.VBox([
        widgets.Label("Create your view"),
        create_view_input, create_view_btn, create_view_output,
        widgets.Label("Query your view"),
        student_query_input, run_student_btn, student_output
    ])
)




#### **Reflection**

- How did you verify that the AI’s draft included all required columns and logic?
- Did the AI’s use of `CASE` and `COALESCE` match your expectations? If not, what did you change?
- Was the AI’s clause order and syntax compatible with SQLite specifically?
- If the AI’s solution ran without errors but gave unexpected results, what debugging steps did you take?
- What’s the risk of over-relying on AI output for multi-requirement SQL queries?




---

# **Question 5**

**Lesson:** Complex Views with Multiple Control Flow Layers

### **Task 5** 

The executive team wants a single permanent SQL view that satisfies the needs of all four stakeholders:

- **Head Chef** — Classify each recipe’s difficulty level.  
- **Marketing Director** — Replace missing descriptions with a placeholder.  
- **CFO** — Label each recipe as `"Budget"` or `"Premium"` based on cost thresholds.  
- **CEO** — Add an `"Attention Needed"` flag for recipes with low ratings **and** missing descriptions.  

**Task**
- Create **one persistent view** that includes **all four logic layers**.
- Use **at least two distinct `CASE` statements** (they can be nested or sequential).
- Incorporate a **null-handling function** where applicable.
  

> ⚠️ **Note:** Focus on structuring your logic so each stakeholder’s requirement is met in a single query.



### **Expected Output Sample for `executive_report` View**

In [ ]:
# Expected output markdown (example for Task 5's view)
expected_output_md = """


| Name                           | Description                          | TotalTime | PrepCategory | RecipeCategory | CostLabel | AttentionNeeded | AvgRating |
| ------------------------------ | ------------------------------------ | --------- | ------------ | -------------- | --------- | --------------- | --------- |
| Picnic Cowboy Caviar           | Make and share this Picnic ...       | 80        | Medium       | Black Beans    | Budget    | No              | 4.0       |
| Turkey Schnitzel With Tots ... | Make and share this Turkey ...       | 40        | Easy         | Poultry        | Premium   | No              | 5.3       |
| Banilla Splash                 | Make and share this Banilla ...      | 6         | Easy         | Beverages      | Budget    | No              | 5.0       |
| Mystery Smoothie               | NULL                                 | 15        | Easy         | Beverages      | Budget    | Yes             | 3.2       |
| Vegan Lasagna                  | Make and share this Vegan Lasagna... | 90        | Hard         | Vegetarian     | Premium   | No              | 4.5       |

"""

expected_output_toggle = widgets.ToggleButton(
    value=False,
    description='Show Expected Output',
    button_style='info',
    icon='eye'
)

expected_output_display = widgets.Output()

def toggle_expected_output(change):
    expected_output_display.clear_output()
    if change['new']:
        expected_output_toggle.description = 'Hide Expected Output'
        with expected_output_display:
            display(Markdown(expected_output_md))
    else:
        expected_output_toggle.description = 'Show Expected Output'

expected_output_toggle.observe(toggle_expected_output, names='value')

display(expected_output_toggle, expected_output_display)

In [ ]:
# Widget to CREATE the view
create_view_input = widgets.Textarea(
    placeholder="Write your SQL to CREATE VIEW here...",
    layout=widgets.Layout(width="100%", height="100px")
)

create_view_btn = widgets.Button(description="Create View", button_style="info")
create_view_output = widgets.Output()

def on_create_view(b):
    create_view_output.clear_output()
    with create_view_output:
        sql = create_view_input.value.strip()
        if not sql:
            print("⚠️ No CREATE VIEW statement entered.")
            return
        try:
            conn.execute(sql)
            print("✅ View created successfully.")
        except Exception as e:
            print(f"❌ SQL Error: {e}")

create_view_btn.on_click(on_create_view)


# Widget to query the created view
student_query_input = widgets.Textarea(
    placeholder="Write your SQL query here (e.g., SELECT * FROM Task2_View)...",
    layout=widgets.Layout(width="100%", height="120px")
)

run_student_btn = widgets.Button(description="Run Your Query", button_style="success")
student_output = widgets.Output()

def run_query(query):
    student_output.clear_output()
    with student_output:
        try:
            if not query.strip():
                print("⚠️ No query entered.")
                return
            result = pd.read_sql_query(query, conn)
            if result.empty:
                print("⚠️ Query ran but returned no results.")
            else:
                display(result.head(10))
        except Exception as e:
            print(f"❌ SQL Error: {e}")

def on_run_student(b):
    run_query(student_query_input.value)

run_student_btn.on_click(on_run_student)


# Display both sections
display(
    widgets.VBox([
        widgets.Label("Create your view"),
        create_view_input, create_view_btn, create_view_output,
        widgets.Label("Query your view"),
        student_query_input, run_student_btn, student_output
    ])
)




#### **Reflection**

- Which stakeholder’s requirement was the most difficult to implement and why?  
- How did you decide where to nest your `CASE` statements versus keeping them separate?  
- Did your null-handling logic interfere with the CEO’s `"Attention Needed"` flag? If yes, how did you fix it?  
- If the CFO later changes the cost threshold, which parts of your view would you need to update?
- If you had used AI for this task, how might it have helped you plan or draft the multi-layer view?
- Would AI have suggested any logic or structure you hadn’t considered? What would you need to **critically evaluate** if you relied on AI output?

Congratulations! 🎉 You’ve completed all five tasks in this series, tackling real-world SQL challenges designed to mirror the workflow of a multi-role executive team.



**What You’ve Achieved**

- Applied **control flow functions** (`CASE`, `COALESCE`) to classify, flag, and handle missing data.  
- Built **persistent views** that consolidate complex logic for multiple stakeholders.  
- Learned to **critically compare AI-generated SQL** with your own queries, spotting differences and validating results.  
- Practiced **null-handling strategies**, nested `CASE` statements, and multi-layered query design.  
- Gained experience thinking like **executives, data engineers, and analysts simultaneously**, balancing business rules with technical logic.  



**Next Steps**

- Review your **queries and reflections**, focusing on decisions where AI could have helped or might have introduced pitfalls.  
- Apply these patterns to new datasets: classify, aggregate, handle nulls, and build executive-ready views.  
- Experiment with **optimizing your queries** while maintaining correctness for multiple stakeholder requirements.  

**Tip:** The interplay between human reasoning and AI assistance is a key skill, AI can suggest solutions, but **critical thinking ensures accuracy, clarity, and alignment with business rules**.


**Self-Assessment Checklist**

In [ ]:
# Self-assessment checklist
checklist_options = [
    "I correctly used CASE statements for multiple stakeholders",
    "I handled NULL values effectively using COALESCE or similar functions",
    "I compared my queries with AI output and reflected critically",
    "I built persistent views consolidating multiple business rules",
    "I understand how query logic affects different executive requirements"
]

checkboxes = [widgets.Checkbox(value=False, description=opt, indent=False) for opt in checklist_options]
output_msg = widgets.Output()

def check_all(change):
    if all(cb.value for cb in checkboxes):
        with output_msg:
            clear_output()
            display(HTML(
                '<b style="color:green">'
                '🎉 Great job! You have completed the self-assessment checklist for this lesson on multi-layer SQL views and null-handling. '
                'You’ve practiced combining control flow logic, handling missing data, and designing executive-ready queries!'
                '</b>'
            ))
    else:
        with output_msg:
            clear_output()

# Attach the observer to all checkboxes
for cb in checkboxes:
    cb.observe(check_all, names='value')

# Display the checkboxes and output message
display(widgets.VBox(checkboxes))
display(output_msg)



#### **Reflection**

- Which stakeholder’s requirement was the most difficult to implement and why?  
- How did you decide where to nest your `CASE` statements versus keeping them separate?  
- Did your null-handling logic interfere with the CEO’s `"Attention Needed"` flag? If yes, how did you fix it?  
- If the CFO later changes the cost threshold, which parts of your view would you need to update?
- If you had used AI for this task, how might it have helped you plan or draft the multi-layer view?
- Would AI have suggested any logic or structure you hadn’t considered? What would you need to **critically evaluate** if you relied on AI output?

Congratulations! 🎉 You’ve completed all five tasks in this series, tackling real-world SQL challenges designed to mirror the workflow of a multi-role executive team.



**What You’ve Achieved**

- Applied **control flow functions** (`CASE`, `COALESCE`) to classify, flag, and handle missing data.  
- Built **persistent views** that consolidate complex logic for multiple stakeholders.  
- Learned to **critically compare AI-generated SQL** with your own queries, spotting differences and validating results.  
- Practiced **null-handling strategies**, nested `CASE` statements, and multi-layered query design.  
- Gained experience thinking like **executives, data engineers, and analysts simultaneously**, balancing business rules with technical logic.  



**Next Steps**

- Review your **queries and reflections**, focusing on decisions where AI could have helped or might have introduced pitfalls.  
- Apply these patterns to new datasets: classify, aggregate, handle nulls, and build executive-ready views.  
- Experiment with **optimizing your queries** while maintaining correctness for multiple stakeholder requirements.  

**Tip:** The interplay between human reasoning and AI assistance is a key skill, AI can suggest solutions, but **critical thinking ensures accuracy, clarity, and alignment with business rules**.


![Done](https://raw.githubusercontent.com/DesmondMokhali/sql-controlflow-views/main/mcq%20round%20table.jpg)


**Self-Assessment Checklist**

In [ ]:
# Self-assessment checklist
checklist_options = [
    "I correctly used CASE statements for multiple stakeholders",
    "I handled NULL values effectively using COALESCE or similar functions",
    "I compared my queries with AI output and reflected critically",
    "I built persistent views consolidating multiple business rules",
    "I understand how query logic affects different executive requirements"
]

checkboxes = [widgets.Checkbox(value=False, description=opt, indent=False) for opt in checklist_options]
output_msg = widgets.Output()

def check_all(change):
    if all(cb.value for cb in checkboxes):
        with output_msg:
            clear_output()
            display(HTML(
                '<b style="color:green">'
                '🎉 Great job! You have completed the self-assessment checklist for this lesson on multi-layer SQL views and null-handling. '
                'You’ve practiced combining control flow logic, handling missing data, and designing executive-ready queries!'
                '</b>'
            ))
    else:
        with output_msg:
            clear_output()

# Attach the observer to all checkboxes
for cb in checkboxes:
    cb.observe(check_all, names='value')

# Display the checkboxes and output message
display(widgets.VBox(checkboxes))
display(output_msg)



### 📝 **Final MCQ Quiz**

Click below to take the quiz:

[👉 Take the Quiz](https://docs.google.com/forms/d/e/1FAIpQLSc12xjhGMeFHk0rOcYAyGVrjsNdBwn6hxYov9U33duuzc-SFg/viewform?usp=dialog)
